<a href="https://colab.research.google.com/github/KyoldFusion/IMDB-Netflix-/blob/main/NetflixIMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [52.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ge

In [2]:
 #import packages

from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType
from pyspark.sql.functions import col, unix_timestamp, to_date
from pyspark.sql.functions import to_timestamp

# we are going to use this to time our queries.
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# Read in data from S3 Bucket
from pyspark import SparkFiles
ratings = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-netflix/ratings.csv"
spark.sparkContext.addFile(ratings)
rf = spark.read.csv(SparkFiles.get("ratings.csv"), sep=",", header=True)

In [4]:
#Create Temp View for ratings.csv
rf.createOrReplaceTempView('ratings')

In [5]:
# Read in data from S3 Bucket
basics = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-netflix/titles_basic.csv"
spark.sparkContext.addFile(basics)
bf = spark.read.csv(SparkFiles.get("titles_basic.csv"), sep=",", header=True)
#bf = bf.withColumn('runtimeminutes', col('runtimeminutes').cast('int'))

In [17]:
#Create Temp View for title_basics.csv
bf.createOrReplaceTempView('basics')
spark.sql("Select * from basics").show(5)

+---+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+-----------------+
|_c0|   tconst|titletype|        primarytitle|       originaltitle|isadult|startyear|endyear|runtimeminutes|           genres|
+---+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+-----------------+
|  0|tt0439997|    movie|           500 Almas|           500 Almas|      0|     2004|   null|         105.0|      Documentary|
|  1|tt0439999|tvSpecial|           80s Mania|           80s Mania|      0|     2001|   null|          50.0|            Music|
|  3|tt0440003|    movie|        A-1 Headline|         A1 tou tiao|      0|     2004|   null|          95.0| Mystery,Thriller|
|  4|tt0440004|  tvMovie| AD/BC: A Rock Opera| AD/BC: A Rock Opera|      0|     2004|   null|          30.0|   Comedy,Musical|
|  5|tt0440008|  tvMovie|Abbamania: We Say...|Abbamania: We Say...|      0|     2004|   null|          50.0|Doc

In [6]:
# Read in data from S3 Bucket
names = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-netflix/filterNames.csv"
spark.sparkContext.addFile(names)
nf = spark.read.csv(SparkFiles.get("filterNames.csv"), sep=",", header=True)

In [8]:
#Create temp view for filterNames.csv
nf.createOrReplaceTempView('names')

In [7]:
#Read in data from S3 bucket
principals = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-netflix/filteredPrincipals.csv"
spark.sparkContext.addFile(principals)
pf = spark.read.csv(SparkFiles.get("filteredPrincipals.csv"), sep=",", header=True)

In [9]:
#Create temp view for FilteredPrincipals.csv
pf.createOrReplaceTempView('principals')

In [ ]:
spark.sql("Select * from principals").show(5)

+----------+--------+---------+--------+----+-----------------+
|    tconst|ordering|   nconst|category| job|       characters|
+----------+--------+---------+--------+----+-----------------+
|tt12189034|       6|nm5943320|   actor|null|      "[""Lee""]"|
|tt12189034|       6|nm5943320|   actor|null|      "[""Lee""]"|
|tt12189034|       6|nm5943320|   actor|null|      "[""Lee""]"|
|tt12189034|       6|nm5943320|   actor|null|      "[""Lee""]"|
|tt12189034|       8|nm7672770|   actor|null|"[""Detective""]"|
+----------+--------+---------+--------+----+-----------------+
only showing top 5 rows



In [ ]:
#Question 1 No Parquet
start_time = time.time()
spark.sql("Select DISTINCT nconst, primaryname, birthyear, deathyear, CASE WHEN birthyear IS NOT NULL and deathyear IS NOT NULL then CAST(deathyear - birthyear AS decimal(3,0)) ELSE COALESCE((birthyear - deathyear), 2021-birthyear) END  Age, CASE WHEN deathyear IS NULL AND birthyear IS NOT NULL THEN 'alive' WHEN deathyear IS NOT NULL THEN 'deceased' WHEN birthyear IS NULL AND deathyear is NULL THEN 'unknown' END livingstatus from names where (primaryprofession LIKE '%actor%' OR primaryprofession LIKE '%actress%') AND birthyear IS NOT NULL").show()
print("---- %s Seconds ---" % (time.time() - start_time))

+---------+--------------------+---------+---------+----+------------+
|   nconst|         primaryname|birthyear|deathyear| Age|livingstatus|
+---------+--------------------+---------+---------+----+------------+
|nm1441958|           Geddy Lee|     1953|     null|68.0|       alive|
|nm1443104|   Wanda Wilkomirska|     1929|     2018|89.0|    deceased|
|nm1450441|         Lexi Lamour|     1976|     null|45.0|       alive|
|nm1451129| Carles Romero Vidal|     1977|     null|44.0|       alive|
|nm1454258|       Cilly Dartell|     1957|     null|64.0|       alive|
|nm6268945|   Monika Potokárová|     1992|     2019|27.0|    deceased|
|nm6276162|    Alasdair Gillies|     1938|     null|83.0|       alive|
|nm6278631|               Spice|     1982|     null|39.0|       alive|
|nm0000139|        Cameron Diaz|     1972|     null|49.0|       alive|
|nm0000457|           John Hurt|     1940|     2017|77.0|    deceased|
|nm0002168|Willeke van Ammel...|     1944|     null|77.0|       alive|
|nm145

In [32]:
#Writing parquet into files
nf.write.parquet('parquet_names', mode='overwrite')

In [33]:
#Read parquet created
npf = spark.read.parquet('parquet_names')

In [34]:
#create temp parquet view for names
npf.createOrReplaceTempView('pnames')

In [ ]:
#Question 1 Parquet
start_time = time.time()
spark.sql("Select DISTINCT nconst, primaryname, birthyear, deathyear, CASE WHEN birthyear IS NOT NULL and deathyear IS NOT NULL then CAST(deathyear - birthyear AS decimal(3,0)) ELSE COALESCE((birthyear - deathyear), 2021-birthyear) END  Age, CASE WHEN deathyear IS NULL AND birthyear IS NOT NULL THEN 'alive' WHEN deathyear IS NOT NULL THEN 'deceased' WHEN birthyear IS NULL AND deathyear is NULL THEN 'unknown'  END LivingStatus from pnames where (primaryprofession LIKE '%actor%' OR primaryprofession LIKE '%actress%') AND birthyear IS NOT NULL").show()
print("---- %s Seconds ---" % (time.time() - start_time))

+---------+--------------------+---------+---------+----+------------+
|   nconst|         primaryname|birthyear|deathyear| Age|LivingStatus|
+---------+--------------------+---------+---------+----+------------+
|nm0937255|      Jan Wlodarczyk|     1934|     2013|79.0|    deceased|
|nm2564430|  Tatiana Grigorieva|     1975|     null|46.0|       alive|
|nm2576486|       Madison Riley|     1990|     null|31.0|       alive|
|nm2576487|      Raymond Alongi|     1960|     null|61.0|       alive|
|nm2589744|  Adriano Aragozzini|     1938|     null|83.0|       alive|
|nm0953941|     Alexandra Zazzi|     1966|     null|55.0|       alive|
|nm0954036|         Natalie Zea|     1975|     null|46.0|       alive|
|nm3747206|     Aleksey Bulatov|     1948|     null|73.0|       alive|
|nm3752434|           Art Kulik|     1984|     null|37.0|       alive|
|nm0990442|          Ben Carson|     1951|     null|70.0|       alive|
|nm3776825|         Andrew Gruz|     1984|     null|37.0|       alive|
|nm262

In [ ]:
spark.sql("Select r.tconst from ratings r join basics b on r.tconst = b.tconst join principals p on p.tconst = b.tconst join names n on p.nconst = n.nconst").show(5)

+---------+
|   tconst|
+---------+
|tt1345836|
|tt1345836|
|tt1345836|
|tt1345836|
|tt1345836|
+---------+
only showing top 5 rows



In [ ]:
spark.sql("Select b.originaltitle, r.averagerating, b.titletype, b.runtimeminutes, b.genres from ratings r join basics b ON b.tconst = r.tconst").show(20)

+--------------------+-------------+---------+--------------+--------------------+
|       originaltitle|averagerating|titletype|runtimeminutes|              genres|
+--------------------+-------------+---------+--------------+--------------------+
|           500 Almas|          7.2|    movie|         105.0|         Documentary|
|         A1 tou tiao|          5.9|    movie|          95.0|    Mystery,Thriller|
| AD/BC: A Rock Opera|          7.4|  tvMovie|          30.0|      Comedy,Musical|
|       Ah ma yau nan|          5.5|    movie|          93.0|        Comedy,Crime|
|      Bau lit do see|          5.4|    movie|          99.0|              Action|
|  The Band Aid Story|          8.1|  tvMovie|          95.0|         Documentary|
|A Beachcombers Ch...|          7.0|  tvMovie|         120.0|Adventure,Comedy,...|
|Blink 182: Punk P...|          4.1|    video|          61.0|         Documentary|
|Boogie special: 5...|          1.0|  tvMovie|          26.0|   Documentary,Music|
|Boo

In [ ]:
bf.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- tconst: string (nullable = true)
 |-- titletype: string (nullable = true)
 |-- primarytitle: string (nullable = true)
 |-- originaltitle: string (nullable = true)
 |-- isadult: string (nullable = true)
 |-- startyear: string (nullable = true)
 |-- endyear: string (nullable = true)
 |-- runtimeminutes: integer (nullable = true)
 |-- genres: string (nullable = true)



In [ ]:
spark.sql("Select runtimeminutes, (runtimeminutes % 60) minutes, (runtimeminutes % 1440 / 60) hours, CONCAT(LEFT(runtimeminutes % 1440 / 60, 1) + RIGHT(runtimeminutes % 1440 / 60, 2)) tes from basics").show(5)

+--------------+-------+------------------+----+
|runtimeminutes|minutes|             hours| tes|
+--------------+-------+------------------+----+
|         105.0|   45.0|              1.75|76.0|
|          50.0|   50.0|0.8333333333333334|34.0|
|          95.0|   35.0|1.5833333333333333|34.0|
|          30.0|   30.0|               0.5| 0.5|
|          50.0|   50.0|0.8333333333333334|34.0|
+--------------+-------+------------------+----+
only showing top 5 rows



In [11]:
spark.sql("Select * from principals").show(5)

+----------+--------+---------+--------+----+-----------------+
|    tconst|ordering|   nconst|category| job|       characters|
+----------+--------+---------+--------+----+-----------------+
|tt12189034|       6|nm5943320|   actor|null|      "[""Lee""]"|
|tt12189034|       6|nm5943320|   actor|null|      "[""Lee""]"|
|tt12189034|       6|nm5943320|   actor|null|      "[""Lee""]"|
|tt12189034|       6|nm5943320|   actor|null|      "[""Lee""]"|
|tt12189034|       8|nm7672770|   actor|null|"[""Detective""]"|
+----------+--------+---------+--------+----+-----------------+
only showing top 5 rows



In [18]:
spark.sql("Select * from basics").show(5)

+---+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+-----------------+
|_c0|   tconst|titletype|        primarytitle|       originaltitle|isadult|startyear|endyear|runtimeminutes|           genres|
+---+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+-----------------+
|  0|tt0439997|    movie|           500 Almas|           500 Almas|      0|     2004|   null|         105.0|      Documentary|
|  1|tt0439999|tvSpecial|           80s Mania|           80s Mania|      0|     2001|   null|          50.0|            Music|
|  3|tt0440003|    movie|        A-1 Headline|         A1 tou tiao|      0|     2004|   null|          95.0| Mystery,Thriller|
|  4|tt0440004|  tvMovie| AD/BC: A Rock Opera| AD/BC: A Rock Opera|      0|     2004|   null|          30.0|   Comedy,Musical|
|  5|tt0440008|  tvMovie|Abbamania: We Say...|Abbamania: We Say...|      0|     2004|   null|          50.0|Doc

In [20]:
spark.sql("Select * from names WHERE primaryprofession LIKE 'producer'").show(5)

+---------+-----------------+---------+---------+-----------------+--------------------+---------+
|   nconst|      primaryname|birthyear|deathyear|primaryprofession|      knownfortitles| knownFor|
+---------+-----------------+---------+---------+-----------------+--------------------+---------+
|nm0460805|    Arlene Knight|     null|     null|         producer|tt4524446,tt40821...|tt4524446|
|nm0462049|Sachiko Kobayashi|     null|     null|         producer|tt5498350,tt00929...|tt6413988|
|nm0459057| Katharina Kleine|     null|     null|         producer|tt0108979,tt01962...|tt0235930|
|nm0459354|Daniel S. Kletzky|     null|     null|         producer|tt0317676,tt01208...|tt0120804|
|nm0459859|     Alex Klippel|     null|     null|         producer|tt0333986,tt02598...|tt0257986|
+---------+-----------------+---------+---------+-----------------+--------------------+---------+
only showing top 5 rows



In [41]:
#Question #3
start_time = time.time()
spark.sql("Select n.primaryname, b.primarytitle, n.knownFor, n.primaryprofession from names n join basics b ON n.knownFor = b.tconst WHERE n.primaryprofession LIKE '%producer%' ").show()
print('--- %s Seconds ---' % (time.time() - start_time))

+--------------------+--------------------+---------+--------------------+
|         primaryname|        primarytitle| knownFor|   primaryprofession|
+--------------------+--------------------+---------+--------------------+
|         Jimmy Smits|Star Wars: Episod...|tt0121766|      actor,producer|
|     Olivia McCallum|Star Wars: Episod...|tt0121766|miscellaneous,ass...|
|       Leigh Ann Fan|Star Wars: Episod...|tt0121766|visual_effects,pr...|
| Eric D. Christensen|Star Wars: Episod...|tt0121766|visual_effects,ex...|
|         Paul Hilton|Star Wars: Episod...|tt0121766|art_department,pr...|
|       Erik A. Baron|Star Wars: Episod...|tt0121766|producer,miscella...|
|         Iain McCaig|Star Wars: Episod...|tt0121766|art_department,pr...|
|         Carl Miller|Star Wars: Episod...|tt0121766|visual_effects,ca...|
| Alexandra Junginger|Star Wars: Episod...|tt0121766|visual_effects,pr...|
|      Fileena Bahris|Star Wars: Episod...|tt0121766|costume_departmen...|
|        Ciara McAvoy|Sta

In [28]:
bf.write.parquet('parquet_basics', mode='overwrite')

In [29]:
bpf = spark.read.parquet('parquet_basics')

In [30]:
bpf.createOrReplaceTempView('bbasics')

In [39]:
#Question 3 Parquet
start_time = time.time()
spark.sql("Select n.primaryname, b.primarytitle, n.knownFor, n.primaryprofession from pnames n join bbasics b ON n.knownFor = b.tconst WHERE n.primaryprofession LIKE '%producer%' ").show()
print('--- %s Seconds ---' % (time.time() - start_time))

+-------------------+--------------------+---------+--------------------+
|        primaryname|        primarytitle| knownFor|   primaryprofession|
+-------------------+--------------------+---------+--------------------+
|         Paul Bader|       Space Pioneer|tt1684722|   producer,director|
|Linda Kaplan Thaler|        Funny People|tt1201167|soundtrack,produc...|
|        Nili Suhami|              August|tt1753460|actress,casting_d...|
|      Daniel Fildes|     Phone Shop Idol|tt6382182|   director,producer|
|     Akeko Yokoyama|            Kamichu!|tt0961173|            producer|
|       Claire Gadea|              Rafiki|tt8286894|producer,miscella...|
|       Claire Gadea|     Shine Your Eyes|tt8477336|producer,miscella...|
|    Jennifer Lunney|On the Road in Am...|tt0954760|production_manage...|
|   Maria Ibrahimova|          Miss Gulag|tt0963993|producer,director...|
|        Jannelle So|    Brown Soup Thing|tt0833878|producer,actress,...|
|      Chris Shearer|  In This Short L